# Setup

In [1]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle
from libHanSolo import unpacker
from matplotlib import pyplot

In [2]:
# Constants
INDEX_ORDER = [(x, x + 1024, x + 2048) for x in range(0, 1024)]
INPUT_SHAPE = (32, 32, 3)

In [3]:
def DataToImage(pixel_values):
    ordered = pixel_values[INDEX_ORDER]
    return np.array(ordered.reshape(INPUT_SHAPE))

def BatchToImages(batch):
    images = unpacker.get_value(batch, 'data')
    return np.array([DataToImage(image) for image in images])

def LabelToVector(label_value):
    arr = np.zeros(10)
    arr[label_value] = 1.0
    return arr

def BatchToLabelVector(batch):
    labels = unpacker.get_value(batch, 'labels')
    return np.array([LabelToVector(label) for label in labels])

def SetupData():
    batches = [unpacker.unpickle(f'./data/data_batch_{index}') for index in range(1, 6)]
    test = unpacker.unpickle(f'./data/test_batch')
    X_train = np.concatenate([BatchToImages(batch) for batch in batches])
    X_test = BatchToImages(test)
    y_train = np.concatenate([BatchToLabelVector(batch) for batch in batches])
    y_test = BatchToLabelVector(test)
    return [X_train, y_train, X_test, y_test]



In [4]:
[X_train, y_train, X_test, y_test] = SetupData()
print(f'{X_train.shape} {X_test.shape} {y_train.shape} {y_test.shape}')

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


# Resnet50 - transfer learning

In [7]:
model = keras.Sequential([
    keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=INPUT_SHAPE, pooling='avg'),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

def process(set):
    return keras.applications.resnet.preprocess_input(set)
    
X_train_processed = process(X_train)
X_test_processed = process(X_test)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_3 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 23,858,890
Trainable params: 23,805,770
Non-trainable params: 53,120
_____________________________________

In [8]:
history_model = model.fit(X_train_processed, y_train, epochs=20, validation_data=(X_test_processed, y_test), batch_size=64)

Epoch 1/20
782/782 [==============================] - 84s 108ms/step - loss: 1.4502 - accuracy: 0.5207 - val_loss: 1.2613 - val_accuracy: 0.5907
Epoch 2/20
782/782 [==============================] - 55s 71ms/step - loss: 1.1434 - accuracy: 0.6299 - val_loss: 1.3355 - val_accuracy: 0.5506
Epoch 3/20
782/782 [==============================] - 49s 63ms/step - loss: 1.1158 - accuracy: 0.6398 - val_loss: 1.0157 - val_accuracy: 0.6638
Epoch 4/20
782/782 [==============================] - 49s 62ms/step - loss: 0.8505 - accuracy: 0.7305 - val_loss: 0.8283 - val_accuracy: 0.7197
Epoch 5/20
782/782 [==============================] - 49s 62ms/step - loss: 0.7214 - accuracy: 0.7706 - val_loss: 0.7854 - val_accuracy: 0.7489
Epoch 6/20
782/782 [==============================] - 49s 62ms/step - loss: 0.6311 - accuracy: 0.7999 - val_loss: 0.7238 - val_accuracy: 0.7593
Epoch 7/20
782/782 [==============================] - 57s 73ms/step - loss: 0.5905 - accuracy: 0.8144 - val_loss: 0.7697 - val_accuracy

# With preprocessing

In [9]:
model_with_preprocessing = keras.Sequential([
    keras.Input(INPUT_SHAPE),
    keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=INPUT_SHAPE, pooling='avg'),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(10, activation="softmax")
])

model_with_preprocessing.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_with_preprocessing.summary()

def process(set):
    return keras.applications.resnet.preprocess_input(set)
    
X_train_processed = process(X_train)
X_test_processed = process(X_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip (RandomFlip)     (None, 32, 32, 3)         0         
_________________________________________________________________
random_rotation (RandomRotat (None, 32, 32, 3)         0         
_________________________________________________________________
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_6 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)               

In [10]:
history_model_preprocessing = model_with_preprocessing.fit(X_train_processed, y_train, epochs=20, validation_data=(X_test_processed, y_test))

Epoch 1/20
1563/1563 [==============================] - 62s 40ms/step - loss: 1.8184 - accuracy: 0.3453 - val_loss: 1.7089 - val_accuracy: 0.3854
Epoch 2/20
1563/1563 [==============================] - 62s 39ms/step - loss: 1.6491 - accuracy: 0.4103 - val_loss: 1.5905 - val_accuracy: 0.4091
Epoch 3/20
1563/1563 [==============================] - 62s 39ms/step - loss: 1.6074 - accuracy: 0.4243 - val_loss: 1.4633 - val_accuracy: 0.4594
Epoch 4/20
1563/1563 [==============================] - 76s 49ms/step - loss: 1.5838 - accuracy: 0.4347 - val_loss: 1.4562 - val_accuracy: 0.4822
Epoch 5/20
1563/1563 [==============================] - 88s 56ms/step - loss: 1.4154 - accuracy: 0.5037 - val_loss: 2.2580 - val_accuracy: 0.2732
Epoch 6/20
1563/1563 [==============================] - 88s 56ms/step - loss: 1.4816 - accuracy: 0.4767 - val_loss: 1.5070 - val_accuracy: 0.4920
Epoch 7/20
1563/1563 [==============================] - 88s 56ms/step - loss: 1.4340 - accuracy: 0.4976 - val_loss: 1.2587 -

In [11]:
model_no_weight = keras.Sequential([
    keras.applications.resnet50.ResNet50(include_top=False, input_shape=INPUT_SHAPE, pooling='avg'),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(10, activation="softmax")
])

model_no_weight.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_no_weight.summary()

def process(set):
    return keras.applications.resnet.preprocess_input(set)
    
X_train_processed = process(X_train)
X_test_processed = process(X_test)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_9 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                650       
Total params: 23,858,890
Trainable params: 23,805,770
Non-trainable params: 53,120
_____________________________________

In [12]:
history_model_no_weight = model_no_weight.fit(X_train_processed, y_train, epochs=20, validation_data=(X_test_processed, y_test))

Epoch 1/20
1563/1563 [==============================] - 61s 39ms/step - loss: 1.7613 - accuracy: 0.3747 - val_loss: 2.5371 - val_accuracy: 0.3116
Epoch 2/20
1563/1563 [==============================] - 61s 39ms/step - loss: 1.4657 - accuracy: 0.4850 - val_loss: 1.4231 - val_accuracy: 0.4829
Epoch 3/20
1563/1563 [==============================] - 61s 39ms/step - loss: 1.3525 - accuracy: 0.5319 - val_loss: 1.1709 - val_accuracy: 0.5831
Epoch 4/20
1563/1563 [==============================] - 60s 39ms/step - loss: 1.2824 - accuracy: 0.5574 - val_loss: 1.7286 - val_accuracy: 0.3812
Epoch 5/20
1563/1563 [==============================] - 86s 55ms/step - loss: 1.2418 - accuracy: 0.5738 - val_loss: 1.0473 - val_accuracy: 0.6354
Epoch 6/20
1563/1563 [==============================] - 86s 55ms/step - loss: 1.1030 - accuracy: 0.6224 - val_loss: 1.0889 - val_accuracy: 0.6406
Epoch 7/20
1563/1563 [==============================] - 86s 55ms/step - loss: 1.1202 - accuracy: 0.6206 - val_loss: 1.2784 -

In [14]:
model.save('data/models/resnet2')
model_with_preprocessing.save('data/models/preprocessed2')
model_no_weight.save('data/models/no-weights')

INFO:tensorflow:Assets written to: data/models/resnet2/assets
INFO:tensorflow:Assets written to: data/models/preprocessed2/assets
INFO:tensorflow:Assets written to: data/models/no-weights/assets
